In [64]:
import keras
import numpy as np
from keras.layers import Add, Dense, Conv2D, BatchNormalization
from keras.layers import Activation, AveragePooling2D, Input, Flatten
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.datasets import cifar10
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.models import Model
from keras.layers import add

In [65]:
batch_size = 32
epochs = 200
n_classes = 10

In [66]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

In [67]:
class Resnet:
    def __init__(self, size=44, stacks=3, starting_filter=16):
        self.size = size
        self.stacks = stacks
        self.starting_filter = starting_filter
        self.residual_blocks = (size - 2) // 6
        
    def get_model(self, input_shape=(32, 32, 3), n_classes=10):
        n_filters = self.starting_filter

        inputs = Input(shape=input_shape)
        network = self.layer(inputs, n_filters)
        network = self.stack(network, n_filters, True)

        for _ in range(self.stacks - 1):
            n_filters *= 2
            network = self.stack(network, n_filters)

        network = Activation('elu')(network)
        network = AveragePooling2D(pool_size=network.shape[1])(network)
        network = Flatten()(network)
        outputs = Dense(n_classes, activation='softmax', 
                        kernel_initializer='he_normal')(network)

        model = Model(inputs=inputs, outputs=outputs)

        return model
    
    def stack(self, inputs, n_filters, first_stack=False):
        stack = inputs

        if first_stack:
            stack = self.identity_block(stack, n_filters)
        else:
            stack = self.convolution_block(stack, n_filters)

        for _ in range(self.residual_blocks - 1):
            stack = self.identity_block(stack, n_filters)

        return stack
    
    def identity_block(self, inputs, n_filters):
        shortcut = inputs

        block = self.layer(inputs, n_filters, normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        block = Add()([shortcut, block])

        return block

    def convolution_block(self, inputs, n_filters, strides=2):
        shortcut = inputs

        block = self.layer(inputs, n_filters, strides=strides,
                           normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        shortcut = self.layer(shortcut, n_filters,
                              kernel_size=1, strides=strides,
                              activation=None, normalize_batch=False)

        block = Add()([shortcut, block])

        return block
    
    def layer(self, inputs, n_filters, kernel_size=3,
              strides=1, activation='elu', normalize_batch=True):
    
        convolution = Conv2D(n_filters, kernel_size=kernel_size,
                             strides=strides, padding='same',
                             kernel_initializer='he_normal',
                             kernel_regularizer=l2(1e-4))

        x = convolution(inputs)

        if normalize_batch:
            x = BatchNormalization()(x)

        if activation is not None:
            x = Activation(activation)(x)

        return x
    
def learning_rate_schedule(epoch):
    learning_rate = 1e-3

    if epoch > 180:
        learning_rate *= 0.5e-3
    elif epoch > 160:
        learning_rate *= 1e-3
    elif epoch > 120:
        learning_rate *= 1e-2
    elif epoch > 80:
        learning_rate *= 1e-1

    return learning_rate

In [68]:
resnet = Resnet()

model = resnet.get_model()

optimizer = Adam(learning_rate_schedule(0))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

model.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_495 (Conv2D)             (None, 32, 32, 16)   448         input_12[0][0]                   
__________________________________________________________________________________________________
batch_normalization_305 (BatchN (None, 32, 32, 16)   64          conv2d_495[0][0]                 
__________________________________________________________________________________________________
activation_313 (Activation)     (None, 32, 32, 16)   0           batch_normalization_305[0][0]    
___________________________________________________________________________________________

In [69]:
lr_scheduler = LearningRateScheduler(learning_rate_schedule)
lr_reducer = ReduceLROnPlateau(factor=0.001, patience=3, min_lr=1e-5)

callbacks = [lr_reducer, lr_scheduler]

datagen = ImageDataGenerator(width_shift_range=0.1,
                             height_shift_range=0.1,
                             rotation_range=20,
                             zoom_range=0.1,
                             horizontal_flip=True)
datagen.fit(x_train)

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
          validation_data=(x_test, y_test),
          epochs=epochs, workers=4,
          callbacks=callbacks)

model.save('model.h5')

Epoch 1/200
1563/1563 [==============================] - 54s 32ms/step - loss: 1.7870 - accuracy: 0.4818 - val_loss: 1.8334 - val_accuracy: 0.5217
Epoch 2/200
1563/1563 [==============================] - 48s 31ms/step - loss: 1.3596 - accuracy: 0.6278 - val_loss: 1.2944 - val_accuracy: 0.6575
Epoch 3/200
1563/1563 [==============================] - 50s 32ms/step - loss: 1.1635 - accuracy: 0.6897 - val_loss: 1.2235 - val_accuracy: 0.6890
Epoch 4/200
1563/1563 [==============================] - 50s 32ms/step - loss: 1.0386 - accuracy: 0.7342 - val_loss: 1.0310 - val_accuracy: 0.7398
Epoch 5/200
1563/1563 [==============================] - 49s 31ms/step - loss: 0.9616 - accuracy: 0.7574 - val_loss: 0.9280 - val_accuracy: 0.7702
Epoch 6/200
1563/1563 [==============================] - 49s 31ms/step - loss: 0.9099 - accuracy: 0.7754 - val_loss: 1.0394 - val_accuracy: 0.7532
Epoch 7/200
1563/1563 [==============================] - 48s 31ms/step - loss: 0.8623 - accuracy: 0.7922 - val_loss: 0

1563/1563 [==============================] - 55s 35ms/step - loss: 0.5884 - accuracy: 0.8800 - val_loss: 0.7056 - val_accuracy: 0.8496
Epoch 57/200
1563/1563 [==============================] - 52s 33ms/step - loss: 0.5820 - accuracy: 0.8812 - val_loss: 0.6556 - val_accuracy: 0.8624
Epoch 58/200
1563/1563 [==============================] - 52s 33ms/step - loss: 0.5841 - accuracy: 0.8802 - val_loss: 0.6675 - val_accuracy: 0.8646
Epoch 59/200
1563/1563 [==============================] - 54s 34ms/step - loss: 0.5829 - accuracy: 0.8815 - val_loss: 0.6364 - val_accuracy: 0.8704
Epoch 60/200
1563/1563 [==============================] - 52s 33ms/step - loss: 0.5850 - accuracy: 0.8804 - val_loss: 0.6350 - val_accuracy: 0.8735
Epoch 61/200
1563/1563 [==============================] - 52s 33ms/step - loss: 0.5788 - accuracy: 0.8825 - val_loss: 0.7414 - val_accuracy: 0.8454
Epoch 62/200
1563/1563 [==============================] - 55s 35ms/step - loss: 0.5803 - accuracy: 0.8799 - val_loss: 0.8319 

1563/1563 [==============================] - 52s 33ms/step - loss: 0.2697 - accuracy: 0.9576 - val_loss: 0.3969 - val_accuracy: 0.9241
Epoch 112/200
1563/1563 [==============================] - 53s 34ms/step - loss: 0.2666 - accuracy: 0.9593 - val_loss: 0.4276 - val_accuracy: 0.9163
Epoch 113/200
1563/1563 [==============================] - 55s 35ms/step - loss: 0.2658 - accuracy: 0.9581 - val_loss: 0.4019 - val_accuracy: 0.9202
Epoch 114/200
1563/1563 [==============================] - 53s 34ms/step - loss: 0.2592 - accuracy: 0.9611 - val_loss: 0.4049 - val_accuracy: 0.9186
Epoch 115/200
1563/1563 [==============================] - 52s 33ms/step - loss: 0.2584 - accuracy: 0.9606 - val_loss: 0.4011 - val_accuracy: 0.9234
Epoch 116/200
1563/1563 [==============================] - 56s 36ms/step - loss: 0.2575 - accuracy: 0.9594 - val_loss: 0.3878 - val_accuracy: 0.9262
Epoch 117/200
1563/1563 [==============================] - 54s 35ms/step - loss: 0.2556 - accuracy: 0.9603 - val_loss: 0

1563/1563 [==============================] - 49s 31ms/step - loss: 0.2038 - accuracy: 0.9754 - val_loss: 0.3923 - val_accuracy: 0.9254
Epoch 167/200
1563/1563 [==============================] - 49s 31ms/step - loss: 0.2037 - accuracy: 0.9743 - val_loss: 0.3881 - val_accuracy: 0.9270
Epoch 168/200
1563/1563 [==============================] - 50s 32ms/step - loss: 0.2033 - accuracy: 0.9753 - val_loss: 0.3900 - val_accuracy: 0.9263
Epoch 169/200
1563/1563 [==============================] - 50s 32ms/step - loss: 0.2031 - accuracy: 0.9754 - val_loss: 0.3873 - val_accuracy: 0.9267
Epoch 170/200
1563/1563 [==============================] - 50s 32ms/step - loss: 0.2048 - accuracy: 0.9746 - val_loss: 0.3907 - val_accuracy: 0.9260
Epoch 171/200
1563/1563 [==============================] - 48s 31ms/step - loss: 0.2062 - accuracy: 0.9739 - val_loss: 0.3889 - val_accuracy: 0.9265
Epoch 172/200
1563/1563 [==============================] - 52s 33ms/step - loss: 0.2035 - accuracy: 0.9746 - val_loss: 0

D:\anaconda\envs\acnn\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
